In [77]:
import json

import matplotlib.pyplot as plt
import numpy as np

In [78]:
with open("bodies.json", "r") as fp:
    zed_data = json.load(fp)

valid_frames = []

for key, value in zed_data.items():
    if value['body_list']:
        valid_frames.append(value)

# COCO 18
joints_name = [
    "NOSE", "NECK", "RIGHT_SHOULDER", "RIGHT_ELBOW", "RIGHT_WRIST", "LEFT_SHOULDER", "LEFT_ELBOW", "LEFT_WRIST", "RIGHT_HIP", "RIGHT_KNEE", "RIGHT_ANKLE", "LEFT_HIP", "LEFT_KNEE", "LEFT_ANKLE", "RIGHT_EYE", "LEFT_EYE", "RIGHT_EAR", "LEFT_EAR"
]
parents = [1, -1, 1, 2, 4, 1, 5, 6, 1, 8, 9, 1, 11, 12, 0, 0, 14, 15]
frametime = 0.05

FIRST_VALID = False

anim_data = []

for idx, frame in enumerate(valid_frames):
    frame_keypoints = frame['body_list'][0]['keypoint']
    if not np.any(np.isnan(frame_keypoints)):
        # Full frame is valid, yay! (which means all keypoints got detected)
        anim_data.append(frame_keypoints)
        FIRST_VALID = True
    else:
        if FIRST_VALID: # we need at least one first fully valid frame... then hopefully we'll interpolate
            anim_data.append(frame_keypoints)
anim_data = np.array(anim_data)
print(anim_data.shape)

In [92]:
FIRST_VALID = False

anim_data2d = []

for idx, frame in enumerate(valid_frames):
    frame_keypoints = frame['body_list'][0]['keypoint_2d']
    if not np.any(np.isnan(frame_keypoints)):
        # Full frame is valid, yay! (which means all keypoints got detected)
        anim_data2d.append(frame_keypoints)
        FIRST_VALID = True
    else:
        if FIRST_VALID: # we need at least one first fully valid frame... then hopefully we'll interpolate
            anim_data2d.append(frame_keypoints)
anim_data2d = np.array(anim_data2d)
print(anim_data2d.shape)

(697, 18, 2)


In [90]:
frame['body_list'][0]['keypoint_2d']

[[-1.0, -1.0],
 [744.79736328125, -48.17592239379883],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [973.8108520507812, -46.3741340637207],
 [1046.57763671875, 313.30377197265625],
 [1202.451416015625, 667.1007690429688],
 [592.84521484375, 587.8441772460938],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [808.658203125, 596.7001342773438],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [-1.0, -1.0],
 [-1.0, -1.0]]

## Interpolate missing data

In [84]:
def interpolate_nan(data):
    """
    Linearly interpolates over NaN values in a 2D array.

    Args:
        data (np.ndarray): Input array of shape (timesteps, channels)
                           with NaN representing missing values.

    Returns:
        np.ndarray: The array with NaN values replaced by interpolated values.
    """
    # Iterate over each channel
    for channel in range(data.shape[1]):
        # Extract the 1D array for this channel
        values = data[:, channel]

        # Identify indices of valid (non-NaN) and invalid (NaN) values
        valid = np.where(~np.isnan(values))[0]
        invalid = np.where(np.isnan(values))[0]

        # Perform linear interpolation only if there are NaNs
        if invalid.size > 0:
            # Interpolate the missing values using np.interp
            values[invalid] = np.interp(invalid, valid, values[valid])

    return data

interp_anim_data = interpolate_nan(anim_data.reshape(anim_data.shape[0], -1))

In [87]:
motion = {
    'joints_name': joints_name,
    'parents': parents,
    'frametime': frametime,
    'anim_data' : interp_anim_data.reshape(anim_data.shape[0], -1, 3).tolist()
}
with open("test_motion.json", "w") as fp:
    json.dump(motion, fp, indent=4)

In [91]:
import cv2
import sys
import pyzed.sl as sl
import ogl_viewer.viewer as gl
import cv_viewer.tracking_viewer as cv_viewer


# Create OpenGL viewer
viewer = gl.GLViewer()
# viewer.init(camera_info.camera_configuration.calibration_parameters.left_cam, body_param.enable_tracking, body_param.body_format)
# Create ZED objects filled in the main loop
bodies = sl.Bodies()
image = sl.Mat()
key_wait = 10

frame_idx = 0
while viewer.is_available():
    # Grab an image
    img = anim_data2d[frame_idx % anim_data2d.shape[0]]
    frame_idx += 1

    # Retrieve left image
    # zed.retrieve_image(image, sl.VIEW.LEFT, sl.MEM.CPU, display_resolution)
    # Retrieve bodies
    # zed.retrieve_bodies(bodies, body_runtime_param)
    # Update GL view
    # viewer.update_view(image, bodies)
    # # Update OCV view
    # image_left_ocv = image.get_data()
    # cv_viewer.render_2D(image_left_ocv, image_scale, bodies.body_list, body_param.enable_tracking, body_param.body_format)
    # cv2.imshow("ZED | 2D View", image_left_ocv)

viewer.exit()
image.free(sl.MEM.CPU)

cv2.destroyAllWindows()

NameError: name 'camera_info' is not defined